<a href="https://colab.research.google.com/github/Ryan-RE-Wang/BCI_project/blob/main/Download_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import seed
from collections import Counter
from datetime import datetime
import time
import os, sys

from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, cohen_kappa_score, accuracy_score

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.5 MB 4.6 MB/s 


In [5]:
sys.path.append('/content/drive/Shareddrives/BCI/sleep_study/sleep_study')
import sleep_study as ss

# Useful Functions

In [6]:
import subprocess

def runcmd(cmd, verbose = False, *args, **kwargs):

    process = subprocess.Popen(
        cmd,
        stdout = subprocess.PIPE,
        stderr = subprocess.PIPE,
        text = True,
        shell = True
    )
    std_out, std_err = process.communicate()
    if verbose:
        print(std_out.strip(), std_err)
    pass

# Download file

In [7]:
ss.init('/content/drive/Shareddrives/BCI/BCI_Data')
print('total number of sleep study files available:', len(ss.data.study_list))

age information stored in /content/age_file.csv
total number of sleep study files available: 0


In [8]:
age_groups = list(zip(range(0, 18), range(1, 19))) + [(18, 100)]

tmp = np.load('/content/drive/Shareddrives/BCI/sleep_study/sleep_study/examples/wavelet/study_lists.npz', allow_pickle=True) 
study_lists = tmp["study_lists"] # filenames that are in each age group

In [ ]:
# this can be parallelized to speed up the process
for i, study_list in enumerate(study_lists):
    if (i < 4):
        continue
    for j, name in enumerate(study_lists[i]):
        if (i == 4 and j <= 156):
            continue

        cmd = 'wget -r -N -c -np --user ryanwang --password ryan57231211 -A "{k}.*" https://physionet.org/files/nch-sleep/3.1.0/Sleep_Data/ -P /content/'.format(k=name)
        runcmd(cmd)

        try:
            features, labels = ss.data.get_demo_wavelet_features_and_labels(name)
            features = np.array(features)
            labels = np.array(labels)

            cmd = 'rm -r /content/physionet.org/'
            runcmd(cmd)

            print( )
            print(features.shape, labels.shape)
            
            fn = '/content/drive/Shareddrives/BCI/BCI_Data/Sleep_Data/wavelet_features' + name
            
            np.savez(fn, labels=labels, features=features)
            print('features from', name, 'i:{}, j:{}'.format(i, j))
            print(' ')
        except:
            continue


(15, 9, 12) (15,)
features from 14338_6229 i:4, j:158
 

(14, 9, 12) (14,)
features from 14377_7231 i:4, j:159
 

(14, 9, 12) (14,)
features from 14410_17341 i:4, j:160
 

(1, 9, 12) (1,)
features from 14533_10999 i:4, j:162
 

(8, 9, 12) (8,)
features from 14677_3190 i:4, j:164
 

(84, 9, 12) (84,)
features from 14707_20176 i:4, j:165
 

(16, 9, 12) (16,)
features from 14710_4837 i:4, j:166
 

(12, 9, 12) (12,)
features from 14722_7546 i:4, j:168
 

(5, 9, 12) (5,)
features from 14863_11794 i:4, j:169
 

(8, 9, 12) (8,)
features from 14968_9133 i:4, j:170
 

(24, 9, 12) (24,)
features from 14971_9346 i:4, j:171
 

(6, 9, 12) (6,)
features from 15034_4834 i:4, j:172
 

(6, 9, 12) (6,)
features from 15064_25915 i:4, j:173
 

(17, 9, 12) (17,)
features from 15208_18769 i:4, j:174
 

(545, 9, 12) (545,)
features from 15220_22624 i:4, j:175
 

(5, 9, 12) (5,)
features from 15409_25495 i:4, j:176
 

(1, 9, 12) (1,)
features from 15442_3055 i:4, j:178
 

(62, 9, 12) (62,)
features from 1566